In [1]:
import pandas as pd
import os
import numpyro
from refit_fvs.data.load import fia_for_diameter_growth_modeling
from refit_fvs.models.fit import fit_wykoff, fit_simpler_wykoff

In [2]:
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.75"
numpyro.set_platform("gpu")
numpyro.set_host_device_count(3)

# Select species
Identify species codes and the FVS variant from which bark ratio coefficients should be drawn.

In [3]:
FVS_ALPHA = "RW"
BARK_VAR = "NC"
FIA_SPCD = 211

In [4]:
BARK_COEFS = "../../data/raw/fvs_barkratio_coefs.csv"
bark = pd.read_csv(BARK_COEFS).set_index(["FIA_SPCD", "FVS_VARIANT"])

bark_b0, bark_b1, bark_b2 = bark.loc[FIA_SPCD, BARK_VAR][["BARK_B0", "BARK_B1", "BARK_B2"]]
bark_b0, bark_b1, bark_b2

(0.0, 0.7012, 1.04862)

# Model-fitting parameters
These don't usually need to be updated when fitting a new species unless you want to add more chains or samples per chain.

In [5]:
NUM_CYCLES = 2
LOC_RANDOM = True
PLOT_RANDOM = True
CHECKPOINT_DIR = f"../../models/maicf/{FVS_ALPHA}/"
NUM_WARMUP = 1000
NUM_SAMPLES = 1000
NUM_CHAINS = 1
CHAIN_METHOD = "parallel"
NUM_BATCHES = 1
SEED = 42
PROGRESS_BAR = True
OVERWRITE = False

In [6]:
data, factors = fia_for_diameter_growth_modeling(
    path="../../data/interim/FIA_remeasured_trees_for_training.csv",
    filter_spp=[FIA_SPCD]
)
obs_variants, obs_locations, obs_plots = factors
data.describe()

,LOCATION,PLOT_ID,FIA_SPCD,MAICF,SLOPE,ASPECT,ELEV,DBH,CR,CR_NEXT,...,INC5YR,INC10YR,MEASYEAR,MEASMON,MEASYEAR_NEXT,MEASMON_NEXT,MEAS_INTERVAL,VAR_IDX,LOC_IDX,PLOT_IDX
count,1444.000000,1.444000e+03,1444.0,1444.000000,1444.000000,1444.000000,1444.000000,1444.000000,1444.000000,1444.000000,...,43.000000,43.000000,1444.000000,1444.000000,1444.000000,1444.000000,1444.0,1444.0,1444.000000,1444.000000
mean,509.353878,6.334137e+10,211.0,180.244191,0.414460,3.134703,8.659280,24.180402,0.462597,0.430727,...,0.400000,0.901163,2005.402355,9.051247,2015.402355,6.430748,10.0,0.0,2.060942,63.712604
std,9.767433,3.307199e+10,0.0,59.105821,0.223303,1.604775,4.004187,17.336063,0.157774,0.148127,...,0.280518,0.615135,2.670289,1.902730,2.670289,1.961528,0.0,0.0,0.805661,37.559892
min,507.000000,6.010155e+10,211.0,67.500000,0.000000,0.000000,1.000000,5.000000,0.050000,0.050000,...,0.050000,0.100000,2001.000000,1.000000,2011.000000,4.000000,10.0,0.0,0.000000,0.000000
25%,507.000000,6.010239e+10,211.0,142.500000,0.250000,2.042035,7.000000,9.675000,0.350000,0.350000,...,0.225000,0.550000,2003.000000,8.000000,2013.000000,5.000000,10.0,0.0,1.000000,25.000000
50%,508.000000,6.010457e+10,211.0,161.226376,0.400000,2.879793,9.000000,24.800000,0.450000,0.400000,...,0.350000,0.750000,2006.000000,10.000000,2016.000000,6.000000,10.0,0.0,2.000000,70.000000
75%,510.000000,6.040815e+10,211.0,221.185120,0.550000,4.310963,11.000000,32.025000,0.550000,0.550000,...,0.450000,0.950000,2008.000000,10.000000,2018.000000,8.000000,10.0,0.0,3.000000,95.000000
max,611.000000,4.102016e+11,211.0,336.576478,1.300000,6.248279,21.000000,191.000000,0.990000,0.950000,...,1.300000,2.550000,2009.000000,12.000000,2019.000000,11.000000,10.0,0.0,3.000000,125.000000


In [7]:
for tree_comp in ['bal', 'relht', 'ballndbh']:
    for stand_comp in ['ba', 'lnba', 'ccf']:
        for pooling in ['unpooled', 'partial']:
            model_name = f"wykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

wykoff_unpooled_mixed_bal-ba
wykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [13:02<00:00,  2.55it/s, 255 steps of size 2.58e-02. a


Done.
Samples saved at ../../models/maicf/RW//wykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/RW//wykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
wykoff_partial_mixed_bal-ba
wykoff_partial_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [22:59<00:00,  1.45it/s, 255 steps of size 2.10e-02. a


Done.
Samples saved at ../../models/maicf/RW//wykoff_partial_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/RW//wykoff_partial_mixed_bal-ba_batch00_mcmc.pkl
wykoff_unpooled_mixed_bal-lnba
wykoff_unpooled_mixed_bal-lnba_batch00


...

Done.
Samples saved at ../../models/maicf/RW//wykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/RW//wykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
wykoff_unpooled_mixed_ballndbh-ccf
wykoff_unpooled_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [09:25<00:00,  3.54it/s, 127 steps of size 2.87e-02. a


Done.
Samples saved at ../../models/maicf/RW//wykoff_unpooled_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/RW//wykoff_unpooled_mixed_ballndbh-ccf_batch00_mcmc.pkl
wykoff_partial_mixed_ballndbh-ccf
wykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [23:16<00:00,  1.43it/s, 255 steps of size 2.56e-02. a


Done.
Samples saved at ../../models/maicf/RW//wykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/RW//wykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl


In [8]:
for pooling in ['unpooled', 'partial']:
    for tree_comp in ['bal', 'relht', 'ballndbh']:
        for stand_comp in ['ba', 'lnba', 'ccf']:
            model_name = f"simplerwykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_simpler_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

simplerwykoff_unpooled_mixed_bal-ba
simplerwykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [10:25<00:00,  3.20it/s, 255 steps of size 2.64e-02. a


Done.
Samples saved at ../../models/maicf/RW//simplerwykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/RW//simplerwykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-lnba
simplerwykoff_unpooled_mixed_bal-lnba_batch00


sample: 100%|█| 2000/2000 [10:21<00:00,  3.22it/s, 255 steps of size 2.67e-02. a


Done.
Samples saved at ../../models/maicf/RW//simplerwykoff_unpooled_mixed_bal-lnba_batch00.pkl
MCMC saved at ../../models/maicf/RW//simplerwykoff_unpooled_mixed_bal-lnba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-ccf
simplerwykoff_unpooled_mixed_bal-ccf_batch00


...

Done.
Samples saved at ../../models/maicf/RW//simplerwykoff_partial_mixed_ballndbh-ba_batch00.pkl
MCMC saved at ../../models/maicf/RW//simplerwykoff_partial_mixed_ballndbh-ba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-lnba
simplerwykoff_partial_mixed_ballndbh-lnba_batch00


sample: 100%|█| 2000/2000 [17:28<00:00,  1.91it/s, 255 steps of size 2.52e-02. a


Done.
Samples saved at ../../models/maicf/RW//simplerwykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/RW//simplerwykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-ccf
simplerwykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [17:19<00:00,  1.92it/s, 255 steps of size 2.06e-02. a


Done.
Samples saved at ../../models/maicf/RW//simplerwykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/RW//simplerwykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl
